In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [2]:
# load and process data set
data_fn = '/Users/mattf/Documents/Spr19/448/final/EyeGEx_retina_combined_genelevel_expectedcounts_byrid_nooutliers.counts.matrix.tsv'
data = pd.read_table(data_fn, sep = '\t', header = 0, index_col = 0)
cells = list(data)
#data # genes are rows. samples are columns
contained = data > 6
data = data[contained.sum(axis=1) >= 91] # Note: 91 when non-MGS samples are included
ind = data.index
col = data.columns
data = np.transpose(data) #????
data = np.log(data + 1)
data = pd.DataFrame(scale(data))
data.index = col
data.columns = ind
#print(list(ind))
#print(col)
#data.rename(index = list(col), columns = list(ind))
#print(data.head())
#data
data.to_csv('processed_expression.csv')


In [3]:
# load metadata
metadata = pd.read_csv('./EyeGEx_meta_combined_inferior_retina_summary_deidentified.txt',names=['r_id','sample_id', 'os_od',
'age', 'sex', 'mgs_level' ,'rin', 'postmortem_interval_hrs'], delim_whitespace=True, index_col = 0)
metadata = metadata.iloc[1:,:]
metadata = metadata.filter(items=cells, axis=0)
metadata.columns=['sample_id', 'os_od', 'age', 'sex', 'MGS Level' ,'rin', 'postmortem_interval_hrs']
labels = metadata['MGS Level']
labels



r_id
100_2    2
101_3    3
102_2    2
103_3    3
104_2    2
105_2    2
106_4    4
107_4    4
109_1    1
11_4     4
110_3    3
111_2    2
112_2    2
113_2    2
114_3    3
115_1    1
116_4    4
117_1    1
119_1    1
120_1    1
121_3    3
122_2    2
123_3    3
124_4    4
125_4    4
126_1    1
127_2    2
128_3    3
129_2    2
130_3    3
        ..
7_3      3
70_3     3
71_2     2
72_2     2
73_4     4
75_2     2
76_3     3
77_2     2
78_3     3
79_3     3
8_2      2
80_2     2
81_1     1
82_2     2
84_2     2
85_2     2
86_2     2
87_2     2
89_1     1
9_1      1
90_2     2
91_2     2
92_3     3
93_2     2
94_4     4
95_4     4
96_3     3
97_2     2
98_3     3
99_1     1
Name: MGS Level, Length: 453, dtype: object

In [4]:
# split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(362, 19346)
(362,)
(91, 19346)
(91,)


In [5]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(X_test, y_test))
clf = LogisticRegression(random_state=0, solver='saga', multi_class='multinomial', penalty='l1', max_iter=100).fit(X_train, y_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(X_test, y_test))

0.26373626373626374
0.2857142857142857


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [6]:
pca = PCA(n_components=20) # reduce the dimensionality of the GENES
pca_samples = pca.fit_transform(data)
Xpca_train, Xpca_test, ypca_train, ypca_test = train_test_split(pca_samples, labels, test_size=0.20, random_state=42)
print(Xpca_train.shape)
print(ypca_train.shape)
print(Xpca_test.shape)
print(ypca_test.shape)

(362, 20)
(362,)
(91, 20)
(91,)


In [12]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(Xpca_train, ypca_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(Xpca_test, ypca_test))

0.34065934065934067


In [11]:
clf = SVC().fit(X_train, y_train)
#print(clf.predict(X_test))
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
clf = SVC().fit(Xpca_train, ypca_train)
#print(clf.predict(Xpca_test))
print(clf.score(Xpca_train, ypca_train))
print(clf.score(Xpca_test, ypca_test))

0.6022099447513812
0.37362637362637363
1.0
0.37362637362637363


In [48]:
# Binary classification
binary_labels = labels!=str(1)



Xb_train, Xb_test, yb_train, yb_test = train_test_split(data, binary_labels, test_size=0.20, random_state=42)
print(Xb_train.shape)
print(yb_train.shape)
print(Xb_test.shape)
print(yb_test.shape)

(362, 19346)
(362,)
(91, 19346)
(91,)


In [41]:
clf = LogisticRegression(random_state=0).fit(Xb_train, yb_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(Xb_test, yb_test))
clf = LogisticRegression(random_state=0, penalty='l1').fit(Xb_train, yb_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(Xb_test, yb_test))

0.26373626373626374
0.7582417582417582


In [44]:
pca = PCA(n_components=20) # reduce the dimensionality of the GENES
pca_samples = pca.fit_transform(data)
Xbpca_train, Xbpca_test, ybpca_train, ybpca_test = train_test_split(pca_samples, binary_labels, test_size=0.20, random_state=42)
print(Xbpca_train.shape)
print(ybpca_train.shape)
print(Xbpca_test.shape)
print(ybpca_test.shape)

(362, 20)
(362,)
(91, 20)
(91,)


In [46]:
clf = LogisticRegression(random_state=0).fit(Xbpca_train, ybpca_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(Xbpca_test, ybpca_test))
clf = LogisticRegression(random_state=0, penalty='l1').fit(Xbpca_train, ybpca_train)
#print(clf.predict(X_test))  # (X[:2, :]) # why :2, I ??
#print(clf.predict_proba(X_test)) #(X[:2, :]) 
print(clf.score(Xbpca_test, ybpca_test))

0.7582417582417582
0.7582417582417582


In [50]:
clf = SVC().fit(Xb_train, yb_train)
#print(clf.predict(X_test))
print(clf.score(Xb_train, yb_train))
print(clf.score(Xb_test, yb_test))
clf = SVC().fit(Xbpca_train, ybpca_train)
#print(clf.predict(Xpca_test))
print(clf.score(Xbpca_train, ybpca_train))
print(clf.score(Xbpca_test, ybpca_test))


0.7679558011049724
0.7802197802197802
1.0
0.7802197802197802
1.0
0.7582417582417582
0.7651933701657458
0.7802197802197802


In [53]:
clf = SVC(kernel='linear').fit(Xb_train, yb_train)
#print(clf.predict(X_test))
print(clf.score(Xb_train, yb_train))
print(clf.score(Xb_test, yb_test))
coef = clf.coef_
print(coef)
#clf = SVC(kernel='linear').fit(Xbpca_train, ybpca_train)
#print(clf.predict(Xpca_test))
#print(clf.score(Xbpca_train, ybpca_train))
#print(clf.score(Xbpca_test, ybpca_test))

1.0
0.7582417582417582
[[ 4.44948167e-04  1.28908974e-03  9.95481965e-05 ...  2.84891501e-03
  -2.70734705e-03  2.96650283e-03]]


In [84]:
argmax = np.argmax(coef)
print(argmax)
print(coef[0][argmax])
argsort = np.argsort(coef)
print(argsort)
print(argsort.shape)
largest = argsort[0][-10:]
print(len(largest))
print(coef[0][largest])
print(data.columns[largest])
print(list(data.columns[largest]))
top = list(data.columns[largest])
top.reverse()
print(top)

17186
0.013178532689861463
[[16716  8348 13135 ... 18653 11859 17186]]
(1, 19346)
10
[0.00988832 0.01008569 0.01023084 0.01025446 0.01039051 0.01060651
 0.01087363 0.01097323 0.01235283 0.01317853]
Index(['ENSG00000173198', 'ENSG00000268555', 'ENSG00000221955',
       'ENSG00000260776', 'ENSG00000253288', 'ENSG00000279208',
       'ENSG00000231212', 'ENSG00000275491', 'ENSG00000178440',
       'ENSG00000260163'],
      dtype='object')
['ENSG00000173198', 'ENSG00000268555', 'ENSG00000221955', 'ENSG00000260776', 'ENSG00000253288', 'ENSG00000279208', 'ENSG00000231212', 'ENSG00000275491', 'ENSG00000178440', 'ENSG00000260163']
['ENSG00000260163', 'ENSG00000178440', 'ENSG00000275491', 'ENSG00000231212', 'ENSG00000279208', 'ENSG00000253288', 'ENSG00000260776', 'ENSG00000221955', 'ENSG00000268555', 'ENSG00000173198']


In [87]:
import mygene
mg = mygene.MyGeneInfo()
l = mg.getgenes(top, fields = 'symbol')
print([d['symbol'] for d in l])

querying 1-10...done.
['AC012508.1', 'LINC00843', 'LINC01730', 'AL590399.3', 'CR381653.2', 'AC046195.1', 'AC104758.3', 'SLC12A8', 'AC123912.4', 'CYSLTR1']


In [95]:
argmax = np.argmax(np.abs(coef))
print(argmax)
print(coef[0][argmax])
argsort = np.argsort(np.abs(coef))
print(argsort)
print(argsort.shape)
largest = argsort[0][-500:]
print(len(largest))
print(coef[0][largest])
print(data.columns[largest])
print(list(data.columns[largest]))
top = list(data.columns[largest])
top.reverse()
print(top)
l = mg.getgenes(top, fields = 'symbol')
print([d['symbol'] for d in l if 'symbol' in d])

17186
0.013178532689861463
[[ 5078 13759  6296 ... 16716 11859 17186]]
(1, 19346)
500
[ 0.00489417 -0.00489735 -0.00489897 -0.00489901  0.00490114  0.00490244
  0.00490361  0.00490769  0.00490874 -0.00490879 -0.00492039  0.00492181
 -0.00492435 -0.00493217  0.00494    -0.00494339 -0.00494745 -0.00495042
 -0.00495207 -0.00495297  0.00495431 -0.00496386  0.00497374 -0.00497622
 -0.00497814 -0.0049782   0.00497866 -0.00498332 -0.00498476  0.00498513
  0.00498584 -0.00498981 -0.00499519  0.00499669 -0.00501444 -0.00502008
 -0.0050213   0.00502251 -0.0050236  -0.00502492  0.00502789 -0.00503419
 -0.00503588  0.00504294 -0.00504569  0.0050466  -0.00504661 -0.00504696
  0.00504852  0.00505385  0.00505561  0.00505876 -0.00506089  0.00506495
 -0.00506772  0.00506832  0.00507094  0.00507102 -0.00507114  0.00507239
 -0.00507797  0.00508388 -0.00509021  0.00509148 -0.00509709  0.00510451
  0.00510768  0.00511315 -0.00511596  0.00511705  0.00512351  0.005124
  0.00513253  0.00513427 -0.00513451 -0.

querying 1-500...done.
['AC012508.1', 'LINC00843', 'PKD1P5', 'MAGED4', 'LINC01730', 'AL590399.3', 'CR381653.2', 'AD000671.1', 'AC046195.1', 'AC104758.3', 'SLC12A8', 'AC123912.4', 'LDHAL6A', 'CYSLTR1', 'YWHAEP1', 'TBC1D3', 'MYH11', 'LINC00839', 'TG', 'AC012181.1', 'AC012414.5', 'PTGES3L-AARSD1', 'AC104135.1', 'OPN1MW', 'RNF125', 'HOXC8', 'LINC02388', 'LINC01411', 'DUXAP9', 'C3orf80', 'AL355315.1', 'AC097382.2', 'AC136632.1', 'LINC01238', 'AC007036.3', 'AL358781.1', 'RN7SL737P', 'BMP4', 'MTATP6P1', 'RSPH10B2', 'LINC02519', 'METTL15P1', 'ATP6V1C2', 'NEUROG2', 'LOC389831', 'HILS1', 'SCIN', 'SH3GL1P2', 'AC106782.2', 'SLC35F2', 'AL138690.1', 'AP000880.1', 'AC010655.4', 'LINC00877', 'POU5F1B', 'ZNF534', 'RAB17', 'CLDND2', 'AC245297.3', 'CCDC163', 'ASNSP3', 'AC242376.1', 'AC105052.4', 'AC004453.1', 'ENPP7P11', 'MIOXP1', 'EMILIN2', 'MKX', 'NOMO3', 'TPTE2P6', 'BX255923.1', 'AP002387.2', 'AC007192.1', 'DMRT2', 'AC100839.2', 'CYMP', 'PWWP3B', 'KLRC1', 'LRRC74B', 'AC090425.2', 'AC073365.1', 'AC0872